In [1]:
import utils
from conversation import ConversationConfig

In [ ]:
utils.load_secrets()

In [3]:
role0_system_prompt = (
    "You are in a conversation with a friend you have a slight crush on"
)
role1_system_prompt = (
    "You are in a conversation with a friend you need to ask for money"
)
role0_start_message = "How are you doing?"

In [4]:
config = ConversationConfig(
    role0_system_prompt=role0_system_prompt,
    role1_system_prompt=role1_system_prompt,
    role0_start_message=role0_start_message,
)
conversation = config.create_conversation()

In [5]:
conversation = config.create_conversation()

In [ ]:
conversation.advance(5)
conversation.print_messages()

In [ ]:
print(
    conversation.analyze(
        "Alice",
        "Bob",
        "You are a Freudian psychotherapist",
        "Analyze the conversation and give it a score from 1 to 10 for how unhinged it is",
    )
)